In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pydicom


In [ ]:
import os
import re
import string
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [ ]:
!pip install -U tensorflow keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [ ]:
import os
import re
import string
import json

def preprocess_text(text):
    """
    Metin ön işleme fonksiyonu. Metni temizler ve normalleştirir.
    """
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text.strip()

def load_txt_data(folder_path):
    """
    TXT dosyalarını yükler ve ön işler.
    """
    reports, labels = [], []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                report = preprocess_text(file.read())
                if report:
                    reports.append(report)
                    labels.append("unknown")  # Etiket mevcut değilse "unknown"

    return reports, labels

def load_jsonl_data(file_path):
    """
    JSONL dosyasını yükler ve ön işler.
    """
    reports, labels = [], []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            report = preprocess_text(data.get('report', ''))
            label = data.get('label', 'unknown')
            if report:
                reports.append(report)
                labels.append(label)

    return reports, labels

def load_all_data(base_folder):
    """
    Tüm veri setlerini yükler ve birleştirir.
    """
    all_reports, all_labels = [], []

    # BIRADS raporlarını yükleme
    birads_base_folder = os.path.join(base_folder, '2000raporBIRADSgruplanmış')
    for subfolder in ['BIRADS1reports(n=50)', 'BIRADS2reports(n=400)', 'BIRADS3reports(n=450)', 'BIRADS4reports(n=550)', 'BIRADS5reports(n=550)']:
        subfolder_path = os.path.join(birads_base_folder, subfolder)
        reports, labels = load_txt_data(subfolder_path)
        all_reports.extend(reports)
        all_labels.extend(labels)

    # NLPJSON klasöründen verileri yükleme
    for i in range(1, 5):
        jsonl_folder = os.path.join(base_folder, 'NLPJSON', str(i))
        jsonl_file = os.path.join(jsonl_folder, 'all.jsonl')
        if os.path.exists(jsonl_file):
            jsonl_reports, jsonl_labels = load_jsonl_data(jsonl_file)
            all_reports.extend(jsonl_reports)
            all_labels.extend(jsonl_labels)

    # NLPTXT-1028 Etiketlenen Raporlar klasöründen verileri yükleme
    nlp_txt_base_folder = os.path.join(base_folder, 'NLPTXT-1028 Etiketlenen Raporlar')
    for subfolder in ['1', '2', '3', '4']:
        subfolder_path = os.path.join(nlp_txt_base_folder, subfolder)
        reports, labels = load_txt_data(subfolder_path)
        all_reports.extend(reports)
        all_labels.extend(labels)

    return all_reports, all_labels

# Verilerin yüklenmesi
base_folder = "/content/drive/MyDrive/Colab Notebooks/VERİLER/NLP-Teknofest24"
reports, labels = load_all_data(base_folder)

print(f"Yüklenen rapor sayısı: {len(reports)}")
print(f"Yüklenen etiket sayısı: {len(labels)}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/VERİLER/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS1reports(n=50)'

In [ ]:
from collections import Counter

def analyze_reports(reports):
    """
    Raporların uzunluklarını ve genel özelliklerini analiz eder.
    """
    lengths = [len(report) for report in reports]
    print(f"Ortalama rapor uzunluğu: {sum(lengths) / len(lengths):.2f} karakter")
    print(f"En kısa rapor uzunluğu: {min(lengths)} karakter")
    print(f"En uzun rapor uzunluğu: {max(lengths)} karakter")

def analyze_labels(labels):
    """
    Etiketlerin dağılımını analiz eder.
    """
    label_counts = Counter(labels)
    print("Etiket Dağılımı:")
    for label, count in label_counts.items():
        print(f"{label}: {count}")

analyze_reports(reports)
analyze_labels(labels)


Ortalama rapor uzunluğu: 514.27 karakter
En kısa rapor uzunluğu: 203 karakter
En uzun rapor uzunluğu: 1214 karakter
Etiket Dağılımı:
unknown: 3028


In [ ]:
import os
import re
import string
import json
import pandas as pd

def preprocess_text(text):
    """
    Metin ön işleme fonksiyonu. Metni temizler ve normalleştirir.
    """
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text.strip()

def extract_birads_label(text):
    """
    Metinden BIRADS etiketini çıkarmak için bir fonksiyon.
    """
    for i in range(1, 6):
        if f'birads{i}' in text:
            return f'BIRADS{i}'
    return 'unknown'

def load_txt_data(folder_path):
    """
    TXT dosyalarını yükler ve ön işler.
    """
    reports, labels = [], []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                report = preprocess_text(file.read())
                if report:
                    label = extract_birads_label(report)
                    reports.append(report)
                    labels.append(label)

    return reports, labels

def load_jsonl_data(file_path):
    """
    JSONL dosyasını yükler ve ön işler.
    """
    reports, labels = [], []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            report = preprocess_text(data.get('report', ''))
            label = extract_birads_label(report)
            if report:
                reports.append(report)
                labels.append(label)

    return reports, labels

def load_all_data(base_folder):
    """
    Tüm veri setlerini yükler ve birleştirir.
    """
    all_reports, all_labels = [], []

    # BIRADS raporlarını yükleme
    birads_base_folder = os.path.join(base_folder, '2000raporBIRADSgruplanmış')
    for subfolder in ['BIRADS1reports(n=50)', 'BIRADS2reports(n=400)', 'BIRADS3reports(n=450)', 'BIRADS4reports(n=550)', 'BIRADS5reports(n=550)']:
        subfolder_path = os.path.join(birads_base_folder, subfolder)
        reports, labels = load_txt_data(subfolder_path)
        all_reports.extend(reports)
        all_labels.extend(labels)

    # NLPJSON klasöründen verileri yükleme
    for i in range(1, 5):
        jsonl_folder = os.path.join(base_folder, 'NLPJSON', str(i))
        jsonl_file = os.path.join(jsonl_folder, 'all.jsonl')
        if os.path.exists(jsonl_file):
            jsonl_reports, jsonl_labels = load_jsonl_data(jsonl_file)
            all_reports.extend(jsonl_reports)
            all_labels.extend(jsonl_labels)

    # NLPTXT-1028 Etiketlenen Raporlar klasöründen verileri yükleme
    nlp_txt_base_folder = os.path.join(base_folder, 'NLPTXT-1028 Etiketlenen Raporlar')
    for subfolder in ['1', '2', '3', '4']:
        subfolder_path = os.path.join(nlp_txt_base_folder, subfolder)
        reports, labels = load_txt_data(subfolder_path)
        all_reports.extend(reports)
        all_labels.extend(labels)

    return all_reports, all_labels

def analyze_data_lengths(reports, labels):
    """
    Raporlar ve etiketler arasındaki uyumsuzlukları kontrol eder.
    """
    if len(reports) != len(labels):
        print(f"UYARI: Rapor sayısı ve etiket sayısı eşleşmiyor!")
        print(f"Rapor sayısı: {len(reports)}")
        print(f"Etiket sayısı: {len(labels)}")

        # Etiketlerin uyumsuzluklarını analiz et
        df = pd.DataFrame({'report': reports, 'label': labels})
        missing_labels = df[df['label'] == 'unknown']
        print(f"Eksik etiketli rapor sayısı: {len(missing_labels)}")
        print("Örnek eksik etiketli raporlar:")
        print(missing_labels.head())
    else:
        print(f"Rapor sayısı ve etiket sayısı eşleşiyor.")
        print(f"Toplam rapor sayısı: {len(reports)}")
        print(f"Toplam etiket sayısı: {len(labels)}")

# Verilerin yüklenmesi
base_folder = "/content/drive/MyDrive/Colab Notebooks/VERİLER/NLP-Teknofest24"
reports, labels = load_all_data(base_folder)

# Verilerin uzunluğunu kontrol etme ve analiz yapma
analyze_data_lengths(reports, labels)


Rapor sayısı ve etiket sayısı eşleşiyor.
Toplam rapor sayısı: 3026
Toplam etiket sayısı: 3026
